In [1]:
from warnings import filterwarnings
filterwarnings("ignore")

In [6]:
Q = open("Desktop/ds/sherlock.txt",encoding="UTF-8")


In [7]:
A = Q.read()

In [8]:
A = A[0:5000]

# Remove all special chars

In [9]:
import re
A1 = re.sub("[\n]"," ",A)
A1 = re.sub("[.,]"," ",A1)
A1 = re.sub("[^a-zA-Z ]","",A1)
A1 = re.sub("   "," ",A1)
A1 = re.sub("  "," ",A1)
A1[0:300]

' Project Gutenbergs The Adventures of Sherlock Holmes by Arthur Conan Doyle This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever You may copy it give it away or reuse it under the terms of the Project Gutenberg License included with this eBook or online '

# Tokenize and case unification

In [10]:
A1 = A1.upper()
A1 = re.sub(" PR","PR",A1)
A1[0:300]

'PROJECT GUTENBERGS THE ADVENTURES OF SHERLOCK HOLMES BY ARTHUR CONAN DOYLE THIS EBOOK IS FOR THE USE OF ANYONE ANYWHERE AT NO COST AND WITH ALMOST NO RESTRICTIONS WHATSOEVER YOU MAY COPY IT GIVE IT AWAY OR REUSE IT UNDER THE TERMS OF THEPROJECT GUTENBERG LICENSE INCLUDED WITH THIS EBOOK OR ONLINE AT'

In [11]:
all_words = A1.split(" ")

# Lets find all unique words

In [12]:
from numpy import unique
unique_words = list(unique(all_words))

# Create X and Y(initial)

In [13]:
len(all_words)

864

In [14]:
X1 = []
Y1 = []
for i in range(4,864,1):
    w1 = all_words[i-4]
    w2 = all_words[i-3]
    w3 = all_words[i-2]
    w4 = all_words[i-1]
    pw = all_words[i]
    X1.append(w1+" "+w2+" "+w3+" "+w4)
    Y1.append(pw)

In [15]:
import pandas as pd
DF = pd.DataFrame([X1,Y1]).T
DF.columns = ["X","Y"]

In [16]:
DF.head()

,X,Y
0,PROJECT GUTENBERGS THE ADVENTURES,OF
1,GUTENBERGS THE ADVENTURES OF,SHERLOCK
2,THE ADVENTURES OF SHERLOCK,HOLMES
3,ADVENTURES OF SHERLOCK HOLMES,BY
4,OF SHERLOCK HOLMES BY,ARTHUR


# Creation of enumerates

In [17]:
lines = list(enumerate(X1))

In [18]:
len(lines)


860

In [19]:
lines[0:10]

[(0, 'PROJECT GUTENBERGS THE ADVENTURES'),
 (1, 'GUTENBERGS THE ADVENTURES OF'),
 (2, 'THE ADVENTURES OF SHERLOCK'),
 (3, 'ADVENTURES OF SHERLOCK HOLMES'),
 (4, 'OF SHERLOCK HOLMES BY'),
 (5, 'SHERLOCK HOLMES BY ARTHUR'),
 (6, 'HOLMES BY ARTHUR CONAN'),
 (7, 'BY ARTHUR CONAN DOYLE'),
 (8, 'ARTHUR CONAN DOYLE THIS'),
 (9, 'CONAN DOYLE THIS EBOOK')]

In [20]:
uni_words_enum = list(enumerate(unique_words))

In [21]:
uni_words_enum[0:10]

[(0, ''),
 (1, 'A'),
 (2, 'ABANDONED'),
 (3, 'ABHORRENT'),
 (4, 'ABSORB'),
 (5, 'ACCOMPLISHED'),
 (6, 'ACCOUNT'),
 (7, 'ACROSS'),
 (8, 'ACTIONS'),
 (9, 'ACTIVITY')]

# Creation of unique words_lookup_dictionary

In [22]:
word_lookup = {}

In [23]:
for i,j in uni_words_enum:
    word_lookup[j]=i

In [24]:
word_lookup['ACTIONS']

8

In [25]:
word_lookup['AND']

27

# Cube declaration

In [27]:
from numpy import zeros
X2 = zeros(shape=(860,426,4)) # lines,uniques,sequence
Y2 = zeros(shape=(860,426))

# Cube formulation(X and Y)

In [28]:
for line_number,line in lines:
    for word_no,word in list(enumerate(line.split(" "))):
        X2[line_number,word_lookup[word],word_no]=1
        #print([line_number,word_no,word_lookup[word]])

In [29]:
#X2  
word_lookup["GUTENBERGS"]

158

In [30]:
X2[0,158,1]

1.0

In [31]:
for i,j in enumerate(DF.Y):
    Y2[i,word_lookup[j]]=1

In [32]:
X2.shape

(860, 426, 4)

In [33]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,LSTM
from keras.activations import softmax
from keras.layers.core import Activation
nn = Sequential()
nn.add(LSTM(426,input_shape=(426, 4)))
nn.add(Dense(426))
nn.add(Activation("softmax"))

In [34]:
nn.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])
nn.fit(X2,Y2,validation_split=0.2,epochs=2)

Epoch 1/2
22/22 [==============================] - 95s 4s/step - loss: 5.8198 - accuracy: 0.0581 - val_loss: 5.7604 - val_accuracy: 0.0407
Epoch 2/2
22/22 [==============================] - 89s 4s/step - loss: 5.4867 - accuracy: 0.0727 - val_loss: 5.9967 - val_accuracy: 0.0407


# CREATE ANN data for output

In [35]:
import numpy as np
Xnew = np.zeros((1,426,4))

In [36]:
W = input("Enter a 3 word sequence to complete: ")
W = W.upper()
for word_no,word in enumerate(W.split(" ")):
    Xnew[0,word_lookup[word],word_no]=1

Enter a 3 word sequence to complete: Sherlock holmes by


In [37]:
len(unique_words)

426

In [39]:
pred = nn.predict(Xnew)

1/1 [==============================] - 0s 178ms/step


In [40]:
T = pd.DataFrame(zip(unique_words,pred[0]),columns=["word","prob"])
T.head()

,word,prob
0,,0.007531
1,A,0.016895
2,ABANDONED,0.001516
3,ABHORRENT,0.001614
4,ABSORB,0.002449


In [41]:
x = T.prob.max()
final_pred = list(T[T.prob == x]["word"])[0]

In [42]:
final_pred

'THE'